# Dropout
## 扰动的稳健性
什么是一个“好”的预测模型？我们期待“好”的预测模型能在未知的数据上有很好的表现。
经典泛化理论认为，为了缩小训练和测试性能之间的差距，应该尽可能使用简单的模型。
简单性可以表现为较小的维度，比如特征的个数；简单性也可以表现为参数的范数；
另外，简单性还可以表现为平滑行，即函数不应该对其输入的微小变化敏感。
例如，当我们对图像进行分类时，我们预计向像素添加一些随机噪声应该是基本无影响的。
1995年，克里斯托弗·毕晓普证明了具有输入噪声的训练等价于Tikhonov正则化 :cite:`Bishop.1995`。
这项工作用数学证实了“要求函数光滑”和“要求函数对输入的随机噪声具有适应性”之间的联系。
然后在2014年，斯里瓦斯塔瓦等人 :cite:`Srivastava.Hinton.Krizhevsky.ea.2014`
就如何将毕晓普的想法应用于网络的内部层提出了一个想法：
在训练过程中，他们建议在计算后续层之前向网络的每一层注入噪声。
因为当训练一个有多层的深层网络时，注入噪声只会在输入-输出映射上增强平滑性。

这个想法被称为*暂退法*（dropout）。
暂退法在前向传播过程中，计算每一内部层的同时注入噪声。
之所以被称为暂退法，是因为我们从表面上看是在训练过程中丢弃（drop out）一些神经元。
那么关键的挑战是如何注入这种噪声。
一种想法是以无偏的方式注入噪声，这样在固定住其他层时，每一层的期望不发生改变。

在标准暂退法正则化中，通过按保留（未丢弃）的节点的分数进行规范化来消除每一层的偏差。
换言之，每个中间活性值$h$以*暂退概率*$p$由随机变量$h'$替换，如下所示：

$$
\begin{aligned}
h' =
\begin{cases}
    0 & \text{ 概率为 } p \\
    \frac{h}{1-p} & \text{ 其他情况}
\end{cases}
\end{aligned}
$$

根据此模型的设计，其期望值保持不变，即$E[h'] = h$。

## 实践中的暂退法
当我们将暂退法应用到隐藏层，以$p$的概率将隐藏单元置为零时，
结果可以看作是一个只包含原始神经元子集的网络。
比如在下图中，删除了$h_2$和$h_5$，
因此输出的计算不再依赖于$h_2$或$h_5$，并且它们各自的梯度在执行反向传播时也会消失。
这样，输出层的计算不能过度依赖于$h_1, \ldots, h_5$的任何一个元素。

![dropout前后的多层感知机](./dropout2-Copy1.svg)

通常，我们在测试时不用暂退法。
给定一个训练好的模型和一个新的样本，我们不会丢弃任何节点
然而也有一些例外：一些研究人员在测试时使用暂退法，
用于估计神经网络预测的“不确定性”：
如果通过许多不同的暂退法遮盖后得到的预测结果都是一致的，那么我们可以说网络发挥更稳定。

## 从零开始实现
### 定义dropout_layer函数
我们从均匀分布$U[0, 1]$中抽取样本，样本数与这层神经网络的维度一致。
然后我们保留那些对应样本大于$p$的节点，把剩下的丢弃。

在下面的代码中，(**我们实现 `dropout_layer` 函数，
该函数以`dropout`的概率丢弃张量输入`X`中的元素**)，
如上所述重新缩放剩余部分：将剩余部分除以`1.0-dropout`。

In [31]:
import torch
import torch.nn as nn
from d2l import torch as d2l

In [32]:
def dropout_layer(X, dropout):
    assert 0 <= dropout <= 1
    # 在本情况下，所有元素都被丢弃
    if dropout == 1:
        return torch.zeros_like(X)
    if dropout == 0:
        return X
    mask = (torch.rand(X.shape) > dropout).float()
    return mask * X / (1 - dropout)

我们可以通过下面几个例子来[**测试`dropout_layer`函数**]。
我们将输入`X`通过暂退法操作，暂退概率分别为0、0.5和1。

In [33]:
X = torch.arange(16, dtype=torch.float32).reshape((2, 8))
X

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])

In [34]:
dropout_layer(X, 1)

tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])

In [35]:
dropout_layer(X, 0)

tensor([[ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.],
        [ 8.,  9., 10., 11., 12., 13., 14., 15.]])

In [36]:
dropout_layer(X, 0.5)

tensor([[ 0.,  0.,  4.,  0.,  0., 10.,  0.,  0.],
        [16., 18., 20.,  0., 24., 26., 28.,  0.]])

### 定义模型参数
同样，我们使用 :numref:`sec_fashion_mnist`中引入的Fashion-MNIST数据集。
我们[**定义具有两个隐藏层的多层感知机，每个隐藏层包含256个单元**]。


In [37]:
num_inputs, num_outputs, num_hiddens1, num_hiddens2 = 784, 10, 256, 256

### 定义模型
我们可以将暂退法应用于每个隐藏层的输出之后，即激活函数之后。
并且可以为每一个层分别设置暂退概率。
常见的技巧是在靠近输入层的地方设置较低的暂退概率。
下面的模型将第一个和第二个隐藏层的暂退概率分别设置为0.2和0.5。
并且暂退法只在训练期间有效。

In [38]:
dropout1 = 0.2 
dropout2 = 0.5

class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs, num_hiddens1, 
                 num_hiddens2, is_training=True):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(num_inputs, num_hiddens1)
        self.fc2 = nn.Linear(num_hiddens1, num_hiddens2)
        self.fc3 = nn.Linear(num_hiddens2, num_outputs)
        self.relu = nn.ReLU()
        self.training = is_training
        self.num_inputs = num_inputs
    
    def forward(self, X):
        H1 = self.relu(self.fc1(X.reshape((-1, self.num_inputs))))  
        if self.training == True:
            H1 = dropout_layer(H1, dropout1)
            
        H2 = self.relu(self.fc2(H1))   
        if self.training == True:
            H2 = dropout_layer(H2, dropout2)
            
        out = self.fc3(H2)
        
        return out
    

net = Net(num_inputs, num_outputs, num_hiddens1, num_hiddens2)        

### 训练


In [39]:
import torch
import torchvision
from torch.utils import data
from torchvision import transforms
trans = transforms.ToTensor()
batch_size = 256
train_data = torchvision.datasets.FashionMNIST(root='../data', train=True, download=True, transform=trans)
test_data = torchvision.datasets.FashionMNIST(root='../data', train=False, download=True, transform=trans)
train_iter = data.DataLoader(train_data, shuffle=True, batch_size=batch_size)
test_iter = data.DataLoader(test_data, shuffle=True, batch_size=batch_size)

In [40]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)

In [41]:
num_epochs = 10
loss_epoch = []  # 分别记录10个epoch的交叉熵损失
correct_epoch = []  # 分别记录10个epoch分类精度
for epoch in range(num_epochs):
    num_samples = 0
    num_batches = 0
    loss_total = 0
    correct_total = 0 
    for X ,y in train_iter:
        y_hat = net(X)
        loss_batch = loss(y_hat, y)
        optimizer.zero_grad()
        loss_batch.backward()
        optimizer.step()
        
        num_samples += len(y)
        num_batches += 1
        loss_total += loss_batch
        correct_total += (torch.argmax(y_hat, axis=1) == y).sum()
    loss_epoch.append(loss_total.item() / num_batches)
    print(f'loss in epoch {epoch + 1}:{loss_total.item() / num_batches}')
    correct_epoch.append(correct_total.item() / num_samples)
    print(f'accuracy in epoch {epoch + 1}:{correct_total.item() / num_samples}')

loss in epoch 1:0.8717734478889627
accuracy in epoch 1:0.6753833333333333
loss in epoch 2:0.5312912311959774
accuracy in epoch 2:0.8056
loss in epoch 3:0.46562048729429856
accuracy in epoch 3:0.8293833333333334
loss in epoch 4:0.4296744813310339
accuracy in epoch 4:0.84375
loss in epoch 5:0.40025618532870677
accuracy in epoch 5:0.8551
loss in epoch 6:0.38625069476188495
accuracy in epoch 6:0.8593666666666666
loss in epoch 7:0.3679675812416888
accuracy in epoch 7:0.8651166666666666
loss in epoch 8:0.35658981647897275
accuracy in epoch 8:0.8686
loss in epoch 9:0.3483600048308677
accuracy in epoch 9:0.87155
loss in epoch 10:0.3374735081449468
accuracy in epoch 10:0.8763333333333333


### 测试

In [42]:
correct_total = 0
num_samples = 0
for X, y in test_iter:
    y_hat = net(X)
    loss_batch = loss(y_hat, y)
    correct_total += (torch.argmax(y_hat, axis=1) == y).sum()
    num_samples += len(y)
accuracy = correct_total / num_samples

In [44]:
accuracy.item()

0.84170001745224

## 简洁实现

### 模型定义与训练

In [48]:
net = nn.Sequential(nn.Flatten(),
        nn.Linear(784, 256),
        nn.ReLU(),
        # 在第一个全连接层之后添加一个dropout层
        nn.Dropout(dropout1),
        nn.Linear(256, 256),
        nn.ReLU(),
        # 在第二个全连接层之后添加一个dropout层
        nn.Dropout(dropout2),
        nn.Linear(256, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, std=0.01)

net.apply(init_weights);

In [49]:
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.5)

In [50]:
num_epochs = 10
loss_epoch = []  # 分别记录10个epoch的交叉熵损失
correct_epoch = []  # 分别记录10个epoch分类精度
for epoch in range(num_epochs):
    num_samples = 0
    num_batches = 0
    loss_total = 0
    correct_total = 0 
    for X ,y in train_iter:
        y_hat = net(X)
        loss_batch = loss(y_hat, y)
        optimizer.zero_grad()
        loss_batch.backward()
        optimizer.step()
        
        num_samples += len(y)
        num_batches += 1
        loss_total += loss_batch
        correct_total += (torch.argmax(y_hat, axis=1) == y).sum()
    loss_epoch.append(loss_total.item() / num_batches)
    print(f'loss in epoch {epoch + 1}:{loss_total.item() / num_batches}')
    correct_epoch.append(correct_total.item() / num_samples)
    print(f'accuracy in epoch {epoch + 1}:{correct_total.item() / num_samples}')

loss in epoch 1:1.0829404790350732
accuracy in epoch 1:0.5761166666666667
loss in epoch 2:0.5645033816073803
accuracy in epoch 2:0.7901
loss in epoch 3:0.4791814276512633
accuracy in epoch 3:0.8231666666666667
loss in epoch 4:0.43804438164893617
accuracy in epoch 4:0.8388
loss in epoch 5:0.40812079247007976
accuracy in epoch 5:0.8501666666666666
loss in epoch 6:0.39328726910530254
accuracy in epoch 6:0.8551333333333333
loss in epoch 7:0.37125438933676863
accuracy in epoch 7:0.8632666666666666
loss in epoch 8:0.35385342861743685
accuracy in epoch 8:0.8707833333333334
loss in epoch 9:0.35049607297207447
accuracy in epoch 9:0.8709166666666667
loss in epoch 10:0.33967018939079124
accuracy in epoch 10:0.8751


### 测试

In [51]:
correct_total = 0
num_samples = 0
for X, y in test_iter:
    y_hat = net(X)
    loss_batch = loss(y_hat, y)
    correct_total += (torch.argmax(y_hat, axis=1) == y).sum()
    num_samples += len(y)
accuracy = correct_total / num_samples

In [52]:
accuracy

tensor(0.8559)